# Binding ALR NCDF

Hans has produced NCDF with CTD data and position. 
I want to interpolate time and depth to T and S and remove NA. Then I'll match the chla bbp from Eco-PUK

All single NCDF will be bind a new big NCDF with segment number as a variable. 

In [1]:
import xarray as xr
import os
import pandas as pd

In [2]:
# list of files in C:\Users\flapet\OneDrive - NOC\Documents\IDAPro\lib\db_building\data\ALR\NCDF_db 
file_list = os.listdir(r"C:\Users\flapet\OneDrive - NOC\Documents\IDAPro\lib\db_building\data\ALR\NCDF_db")
full_file_list = [os.path.join(r"C:\Users\flapet\OneDrive - NOC\Documents\IDAPro\lib\db_building\data\ALR\NCDF_db", f) for f in file_list if f.endswith('.nc')]

In [3]:
def add_segment_id(ds):
    # get the last part and take its first character
    letter = ds.attrs['event_type']
    number = ds.attrs['unique_id']
    number2 = ds.attrs['segment_id']

    event_direction_dict = {'x': 'transition', 'd': 'descent', 's': 'surface', 'u': 'ascent'} 

    event_direction = event_direction_dict.get(letter)
    event_id = f"{event_direction}_{number}"

    ds['event_id'] = event_id
    ds['segment_number'] = number2
    ds['event_direction'] = event_direction

    return ds

In [4]:
ds = xr.open_mfdataset(
    full_file_list,
    preprocess= add_segment_id
)

In [5]:
# Make TIME a single chunk
ds = ds.chunk(dict(TIME=-1))

# Now interpolation works
ds["LAT"] = ds["LAT"].interpolate_na(dim="TIME", method="linear")
ds["LON"] = ds["LON"].interpolate_na(dim="TIME", method="linear")

# Remove rows where Temperature is NA
ds = ds.dropna(dim='TIME', subset=['Temperature'])

In [6]:
display(ds)

<xarray.Dataset> Size: 118MB
Dimensions:           (TIME: 844630)
Coordinates:
  * TIME              (TIME) datetime64[ns] 7MB 2024-06-12T00:08:26.395245056...
Data variables:
    DEPTH_SCI         (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    DEPTH_ENG         (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    DEPTH_SCI_INTERP  (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    LAT               (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    LON               (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    Temperature       (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    Conductivity      (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    Oxygen            (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    event_id          (TIME) <U14 47MB dask.array<chunksize=(844630,), meta=np.ndarray>
    segment_number    (TIME) int32 3MB dask.array<chunksize=(844630,), meta=np.ndarray>
    event_direction   (TIME) object 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
Attributes: (12/14)
    Conventions:          CF-1.9
    title:                ALR profile/transect/other export
    institution:          NOCS
    source:               Polars→xarray export
    history:              Created by export_alr_netcdf()
    author:               hans.hilder@noc.ac.uk
    ...                   ...
    segment_id:           0
    event_type:           x
    group_kind:           other
    group_key:            [0 1]
    surface_threshold_m:  5.0
    event_start_utc:      2024-06-11T23:53:13.455329+00:00

## Adding ecopuk data

In [7]:
eco_puk_file = "../output/ALR6_fluorometer_data.csv"

In [8]:
eco_puk_dat = pd.read_csv(eco_puk_file)
eco_puk_dat['timestamp'] = pd.to_datetime(eco_puk_dat['nav_timestamp'], unit = "s")
eco_puk_dat = eco_puk_dat.set_index('timestamp')
eco_puk_dat = eco_puk_dat.drop(columns=["datetime"])


# Convert to xarray Dataset
eco_puk_dat_xr = eco_puk_dat.to_xarray()

In [9]:
import numpy as np

In [10]:
# get timestamp values as a pandas Index
timestamps = pd.Index(eco_puk_dat_xr["timestamp"].values)

# find unique timestamps
_, idx = np.unique(timestamps, return_index=True)

# select only unique timestamps
eco_puk_dat_xr_unique = eco_puk_dat_xr.isel(timestamp=idx)

In [11]:
display(eco_puk_dat_xr_unique)

<xarray.Dataset> Size: 44MB
Dimensions:        (timestamp: 779061)
Coordinates:
  * timestamp      (timestamp) datetime64[ns] 6MB 2024-06-11T14:05:59.9525339...
Data variables:
    nav_timestamp  (timestamp) float64 6MB 1.718e+09 1.718e+09 ... 1.719e+09
    lon            (timestamp) float64 6MB -20.27 -20.27 ... -20.13 -20.13
    lat            (timestamp) float64 6MB 63.35 63.35 63.35 ... 60.2 60.2 60.2
    depth          (timestamp) float64 6MB 0.05963 0.05963 ... 0.09076 0.0964
    fluo           (timestamp) float64 6MB 0.6205 0.584 0.6716 ... 1.854 1.84
    beta           (timestamp) float64 6MB 0.0002599 0.0003173 ... 0.0004684

In [12]:
df_xr_on_ds = eco_puk_dat_xr_unique.rename({'timestamp': 'TIME'}).reindex(
    TIME=ds.TIME,
    method="nearest",
    tolerance=pd.Timedelta("5s")  # safer to use pandas Timedelta
)



In [13]:
ds_merged = xr.merge([ds, df_xr_on_ds])


In [97]:
display(ds_merged)


<xarray.Dataset> Size: 155MB
Dimensions:           (TIME: 844630)
Coordinates:
  * TIME              (TIME) datetime64[ns] 7MB 2024-06-12T00:08:26.395245056...
Data variables: (12/17)
    DEPTH_SCI         (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    DEPTH_ENG         (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    DEPTH_SCI_INTERP  (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    LAT               (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    LON               (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    Temperature       (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    ...                ...
    nav_timestamp     (TIME) float64 7MB 1.718e+09 1.718e+09 ... 1.719e+09
    lon               (TIME) float64 7MB -20.27 -20.27 -20.27 ... -20.13 -20.13
    lat               (TIME) float64 7MB 63.35 63.35 63.35 ... 60.2 60.2 60.2
    depth             (TIME) float64 7MB 0.5227 0.5227 0.5232 ... 0.0964 0.0964
    fluo              (TIME) float64 7MB 0.3577 0.3504 0.3358 ... 1.84 1.84
    beta              (TIME) float64 7MB 0.0002062 0.000208 ... 0.0004684
Attributes: (12/14)
    Conventions:          CF-1.9
    title:                ALR profile/transect/other export
    institution:          NOCS
    source:               Polars→xarray export
    history:              Created by export_alr_netcdf()
    author:               hans.hilder@noc.ac.uk
    ...                   ...
    segment_id:           0
    event_type:           x
    group_kind:           other
    group_key:            [0 1]
    surface_threshold_m:  5.0
    event_start_utc:      2024-06-11T23:53:13.455329+00:00

In [14]:
clean_eco_xr = ds_merged.dropna(dim='TIME', subset=['fluo'])

In [15]:
display(clean_eco_xr)

<xarray.Dataset> Size: 159MB
Dimensions:           (TIME: 844630)
Coordinates:
  * TIME              (TIME) datetime64[ns] 7MB 2024-06-12T00:08:26.395245056...
Data variables: (12/17)
    DEPTH_SCI         (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    DEPTH_ENG         (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    DEPTH_SCI_INTERP  (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    LAT               (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    LON               (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    Temperature       (TIME) float64 7MB dask.array<chunksize=(844630,), meta=np.ndarray>
    ...                ...
    nav_timestamp     (TIME) float64 7MB 1.718e+09 1.718e+09 ... 1.719e+09
    lon               (TIME) float64 7MB -20.27 -20.27 -20.27 ... -20.13 -20.13
    lat               (TIME) float64 7MB 63.35 63.35 63.35 ... 60.2 60.2 60.2
    depth             (TIME) float64 7MB 0.5227 0.5227 0.5232 ... 0.0964 0.0964
    fluo              (TIME) float64 7MB 0.3577 0.3504 0.3358 ... 1.84 1.84
    beta              (TIME) float64 7MB 0.0002062 0.000208 ... 0.0004684
Attributes: (12/14)
    Conventions:          CF-1.9
    title:                ALR profile/transect/other export
    institution:          NOCS
    source:               Polars→xarray export
    history:              Created by export_alr_netcdf()
    author:               hans.hilder@noc.ac.uk
    ...                   ...
    segment_id:           0
    event_type:           x
    group_kind:           other
    group_key:            [0 1]
    surface_threshold_m:  5.0
    event_start_utc:      2024-06-11T23:53:13.455329+00:00

In [16]:
clean_eco_xr.to_netcdf("../data/ALR/clean_eco_xr_output.nc", format="NETCDF4")

C:\Users\flapet\AppData\Local\Temp\ipykernel_27164\2543349860.py:1: SerializationWarning: variable event_direction has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  clean_eco_xr.to_netcdf("../data/ALR/clean_eco_xr_output.nc", format="NETCDF4")
